In [230]:
DATABASE_HOST="172.21.0.8"
# DATABASE_HOST="172.20.0.3"
DATABASE_DATABASE="fabricexplorer"
DATABASE_USERNAME="hppoc"
DATABASE_PASSWORD="password"
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import bamboolib as bam
# import hiplot as hip
import matplotlib.pyplot as ptlib
import plotly.express as px
import json

# to get the ips of docker containers
# docker ps -q | xargs -n 1 docker inspect --format '{{ .Name }} {{range .NetworkSettings.Networks}} {{.IPAddress}}{{end}}' | sed 's#^/##';
# list all user defined tables and schemas
# "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"

#initializing resources
engine = create_engine('postgresql://'+DATABASE_USERNAME+':'+DATABASE_PASSWORD+'@'+DATABASE_HOST+':5432/'+DATABASE_DATABASE)

txQuery = "SELECT * FROM transactions"
txDf = pd.read_sql(txQuery,con=engine)
# txDf.to_csv('backup.csv')
txDf=txDf[txDf['read_set'].isna()==False]
txDf=txDf[txDf['chaincodename'].str.contains('_lifecycle')==False]
txDf=txDf[txDf['type'].str.contains('CONFIG')==False]
txDf=txDf.drop(columns=["txhash","chaincode_id","endorser_msp_id","type","createdt","channel_genesis_hash","envelope_signature","creator_id_bytes", "creator_nonce", "payload_extension","tx_response","payload_proposal_hash","endorser_id_bytes","endorser_signature", "network_name"])
txDf

    id  blockid chaincodename  status creator_msp_id  \
0    6        7         basic   200.0        Org1MSP   
7   13        6         basic   200.0        Org1MSP   
8   14        8         basic   200.0        Org2MSP   
9   15        9         basic   200.0        Org2MSP   
10  16       10         basic   200.0        Org2MSP   
11  17       11         basic   200.0        Org2MSP   
12  18       12         basic   200.0        Org2MSP   
13  19       13         basic   200.0        Org2MSP   
14  20       14         basic   200.0        Org2MSP   
15  21       15         basic   200.0        Org2MSP   
16  22       16         basic   200.0        Org2MSP   
17  23       16         basic   200.0        Org2MSP   
18  24       17         basic   200.0        Org2MSP   
19  25       17         basic   200.0        Org2MSP   
20  26       18         basic   200.0        Org2MSP   
21  27       19         basic   200.0        Org2MSP   
22  28       20         basic   200.0        Org

In [231]:
read_set=txDf["read_set"]
write_set=txDf["write_set"]
for item in read_set:
    item.pop(0)
for item in write_set:
    item.pop(0)

In [232]:
txDf['read_set']=read_set
txDf['write_set']=write_set
txDf = txDf.explode('read_set')
txDf = txDf.explode('write_set')
txDf
# txDf = pd.pivot(txDf, columns=['id'], index=['read_set'])
# txDf

    id  blockid chaincodename  status creator_msp_id  \
0    6        7         basic   200.0        Org1MSP   
7   13        6         basic   200.0        Org1MSP   
8   14        8         basic   200.0        Org2MSP   
9   15        9         basic   200.0        Org2MSP   
10  16       10         basic   200.0        Org2MSP   
11  17       11         basic   200.0        Org2MSP   
12  18       12         basic   200.0        Org2MSP   
13  19       13         basic   200.0        Org2MSP   
14  20       14         basic   200.0        Org2MSP   
15  21       15         basic   200.0        Org2MSP   
16  22       16         basic   200.0        Org2MSP   
17  23       16         basic   200.0        Org2MSP   
18  24       17         basic   200.0        Org2MSP   
19  25       17         basic   200.0        Org2MSP   
20  26       18         basic   200.0        Org2MSP   
21  27       19         basic   200.0        Org2MSP   
22  28       20         basic   200.0        Org

In [233]:
txDf['r_chaincode'] =txDf['read_set'].apply(lambda x: x['chaincode'])
txDf['rs'] =txDf['read_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['chaincodename','read_set'])
txDf['w_chaincode'] =txDf['write_set'].apply(lambda x: x['chaincode'])
txDf['ws'] =txDf['write_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['write_set'])
txDf

    id  blockid  status creator_msp_id     validation_code  \
0    6        7   200.0        Org1MSP               VALID   
7   13        6   200.0        Org1MSP               VALID   
8   14        8   200.0        Org2MSP               VALID   
9   15        9   200.0        Org2MSP               VALID   
10  16       10   200.0        Org2MSP               VALID   
11  17       11   200.0        Org2MSP               VALID   
12  18       12   200.0        Org2MSP               VALID   
13  19       13   200.0        Org2MSP               VALID   
14  20       14   200.0        Org2MSP               VALID   
15  21       15   200.0        Org2MSP               VALID   
16  22       16   200.0        Org2MSP               VALID   
17  23       16   200.0        Org2MSP  MVCC_READ_CONFLICT   
18  24       17   200.0        Org2MSP               VALID   
19  25       17   200.0        Org2MSP  MVCC_READ_CONFLICT   
20  26       18   200.0        Org2MSP               VALID   
21  27  

In [234]:
def classifyWrite(ws):
    if str(ws)=='nan':
        return np.nan
    elif ws['is_delete']:
        return 'DELETE'
    else:
        return 'WRITE'
txDfr = txDf.drop(columns=['w_chaincode','ws']).explode('rs')
txDfr = txDfr.rename(columns={'r_chaincode': 'chaincode','rs':'key'})
txDfr['access_type'] = txDfr['key'].apply(lambda x: 'READ')
txDfr['version_block'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['block_num'])
txDfr['version_tx'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['tx_num'])
txDfw = txDf.drop(columns=['r_chaincode','rs']).explode('ws')
txDfw = txDfw.rename(columns={'w_chaincode': 'chaincode','ws':'key'})
txDfw['access_type'] = txDfw['key'].apply(lambda x: classifyWrite(x))
txDf = txDfr.append(txDfw).reset_index(drop=True).rename(columns={'id':'txid'})
txDf['key']=txDf['key'].apply(lambda x:np.nan if str(x) == 'nan' else x['key'])

In [238]:
txDf

    txid  blockid  status creator_msp_id     validation_code  \
0      6        7   200.0        Org1MSP               VALID   
1     13        6   200.0        Org1MSP               VALID   
2     14        8   200.0        Org2MSP               VALID   
3     15        9   200.0        Org2MSP               VALID   
4     16       10   200.0        Org2MSP               VALID   
5     17       11   200.0        Org2MSP               VALID   
6     18       12   200.0        Org2MSP               VALID   
7     19       13   200.0        Org2MSP               VALID   
8     20       14   200.0        Org2MSP               VALID   
9     21       15   200.0        Org2MSP               VALID   
10    22       16   200.0        Org2MSP               VALID   
11    23       16   200.0        Org2MSP  MVCC_READ_CONFLICT   
12    24       17   200.0        Org2MSP               VALID   
13    25       17   200.0        Org2MSP  MVCC_READ_CONFLICT   
14    26       18   200.0        Org2MSP

In [236]:
query = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"
# query = "select * from peer"
idkdf = pd.read_sql(query,con=engine)
idkdf

  schemaname           tablename tableowner tablespace  hasindexes  hasrules  \
0     public              blocks      hppoc       None        True     False   
1     public          chaincodes      hppoc       None        True     False   
2     public  peer_ref_chaincode      hppoc       None        True     False   
3     public             channel      hppoc       None        True     False   
4     public                peer      hppoc       None        True     False   
5     public    peer_ref_channel      hppoc       None        True     False   
6     public             orderer      hppoc       None        True     False   
7     public        transactions      hppoc       None        True     False   
8     public               users      hppoc       None        True     False   
9     public          write_lock      hppoc       None        True     False   

   hastriggers  rowsecurity  
0        False        False  
1        False        False  
2        False        False  